# This file is based on the data of hw3
## We mainly do two things:
-  Classify data
- Train the regression model



In [1]:
# check python version
# must be 3.8 to 3.10
from platform import python_version

print(python_version())

3.10.8


In [2]:
# install packages
!pip install polygon-api-client
!pip install sqlalchemy
!pip install pandas
!pip install matplotlib
!pip install numpy==1.22.3
!pip install pycaret

  Using cached polygon_api_client-1.4.0-py3-none-any.whl (34 kB)
  Using cached certifi-2022.9.24-py3-none-any.whl (161 kB)
  Using cached websockets-10.4-cp310-cp310-macosx_10_9_x86_64.whl (97 kB)
  Using cached urllib3-1.26.13-py2.py3-none-any.whl (140 kB)
You should consider upgrading via the '/Users/tangyuqi/Documents/Data Engineering/DataEngineering-HW4/venv/bin/python -m pip install --upgrade pip' command.
  Using cached SQLAlchemy-1.4.44-cp310-cp310-macosx_10_15_x86_64.whl (1.6 MB)
  Using cached greenlet-2.0.1-cp310-cp310-macosx_10_15_x86_64.whl (203 kB)
You should consider upgrading via the '/Users/tangyuqi/Documents/Data Engineering/DataEngineering-HW4/venv/bin/python -m pip install --upgrade pip' command.
  Using cached pandas-1.5.2-cp310-cp310-macosx_10_9_x86_64.whl (12.0 MB)
  Using cached pytz-2022.6-py2.py3-none-any.whl (498 kB)
     |████████████████████████████████| 18.1 MB 19.8 MB/s            
You should consider upgrading via the '/Users/tangyuqi/Documents/Data Engi